In [1]:
# The MIT License (MIT) Copyright (c) 2023 Emilio Morales
#
# Permission is hereby granted, free of charge, to any person obtaining a copy of 
# this software and associated documentation files (the "Software"), to deal in the Software without 
# restriction, including without limitation the rights to use, copy, modify, merge, publish, 
# distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the 
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all copies or 
# substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, 
# INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND 
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES 
# OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN 
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/milmor/NLP/blob/main/Notebooks/12_RNN_LSTM.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
</table>

# Generación de texto
  
- Game of thrones book: https://www.kaggle.com/datasets/khulasasndh/game-of-thrones-books

In [2]:
import torch
import pandas as pd

torch.__version__

'2.0.1+cu117'

In [3]:
torch.manual_seed(77)

## 1.- Conjuntos de datos

In [4]:
path = './001ssb.txt'
book = open(path, 'rb').read().decode(encoding='utf-8').lower()

print(f'Words: {len(book)}')

Words: 1628063


In [5]:
import re

words = re.findall(r'\b\w+\b|[\.,;!?()"\']', book)

maxlen = 50
# Crear lotes de 50 palabras
sentences = [words[i:i + maxlen] for i in range(0, len(words), maxlen)]

In [6]:
sentences[0][:20]

['a',
 'game',
 'of',
 'thrones',
 'book',
 'one',
 'of',
 'a',
 'song',
 'of',
 'ice',
 'and',
 'fire',
 'by',
 'george',
 'r',
 '.',
 'r',
 '.',
 'martin']

In [7]:
len(sentences)

7367

## 2.- Pipeline

- Crea vocabulario y define tokenizer.

In [8]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab as Vocab
from collections import Counter

In [9]:
tokenizer = get_tokenizer('basic_english')

In [10]:
def build_vocab(text, tokenizer, min_freq=5):
    counter = Counter()
    for string_ in text:
        # join words list into single string
        string_ = ' '.join(string_)
        counter.update(tokenizer(string_))
    return Vocab(counter, min_freq=min_freq,
                 specials=['<unk>', '<pad>'])

vocab = build_vocab(sentences, tokenizer, 0)

In [11]:
vocab_size = len(vocab)
vocab_size

11916

In [12]:
vocab.get_itos()[0] 

'<unk>'

In [13]:
maxlen = 64

def data_process(text):
    data = []
    for raw_txt in text:
        # join words list into single string
        raw_txt = ' '.join(raw_txt)
        tensor_ = torch.tensor([vocab[token] for token in tokenizer(raw_txt)],
                                dtype=torch.long)
        if tensor_.shape[0] < maxlen:
            x = tensor_[:-1]
            y = tensor_[1:]
            data.append((x, y))
    return data

train_data = data_process(sentences)

In [14]:
batch_size = 64
PAD_IDX = vocab['<pad>']

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
    x, y = [], []
    for (x_item, y_item) in data_batch:
        x.append(x_item)
        y.append(y_item)

    x = pad_sequence(x, batch_first=True, padding_value=PAD_IDX)
    y = pad_sequence(y, batch_first=True, padding_value=PAD_IDX)
    return x, y


train_loader = DataLoader(train_data, batch_size=batch_size,
                          shuffle=True, collate_fn=generate_batch, 
                          num_workers=4, pin_memory=True)

In [15]:
%%timeit
train_batch, target_batch = next(iter(train_loader))

110 ms ± 3.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
train_batch, target_batch = next(iter(train_loader))

In [17]:
train_batch.shape, target_batch.shape

(torch.Size([64, 49]), torch.Size([64, 49]))

## 3.- Modelo

In [18]:
import torch.nn as nn
from torch import optim
import time

In [19]:
class RNN(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, model_dim=128):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.rnn = nn.LSTM(input_size=emb_dim, 
                        hidden_size=model_dim, 
                        num_layers=1, 
                        batch_first=True)
        self.fc1 = nn.Linear(model_dim, vocab_size)
        
    def forward(self, x):
        x = self.embedding(x)
        x, (hidden, cell) = self.rnn(x)
        x = self.fc1(x)
        return x

rnn = RNN(vocab_size)
output_batch = rnn(train_batch)
output_batch.shape, target_batch.shape

(torch.Size([64, 49, 11916]), torch.Size([64, 49]))

## 4.- Entrenamiento

In [20]:
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [21]:
def train(model, device, train_loader, optimizer, epoch):
    start = time.time()
    running_loss = 0.0
    model.train()
    for inputs, labels in train_loader:
        labels = labels.view(-1)
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        outputs = outputs.view(-1, outputs.size(-1))
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return f'\nTime for epoch {epoch} is {time.time()-start:.4f} sec Train loss: {running_loss / len(train_loader):.4f}'

In [22]:
def generate(model, sentence, device, maxlen):
    with torch.no_grad():
        model.eval()
        idx = torch.tensor([vocab[token] for token in tokenizer(sentence)],
                                    dtype=torch.long)
        idx = idx.reshape([1, -1])
        maxlen = maxlen - idx.shape[-1]

        for _ in range(maxlen):
            idx = idx.to(device)
            logits = model(idx)[:, -1, :]      
            probs = torch.softmax(logits, dim=-1)

            _, idx_next = torch.topk(probs, k=1, dim=-1)
            idx = torch.cat((idx, idx_next), dim=1)

        txt = " ".join(
                    [vocab.get_itos()[idx[0, _]] for _ in range(maxlen)]
                )
    return txt

start = 'tyrion'
#generate(rnn, start, 'cpu', maxlen)

In [23]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [24]:
rnn.to(device)
optimizer = optim.Adam(rnn.parameters(), lr=0.001)

In [25]:
epochs = 200

for epoch in range(epochs):
    report = train(rnn, device, train_loader, optimizer, epoch)
    if epoch % 20 == 0:
        print(report)
        generated_text = generate(rnn, start, device, maxlen)
        print(f'Output: {generated_text}')


Time for epoch 0 is 0.9381 sec Train loss: 6.8471
Output: tyrion to the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the , the

Time for epoch 20 is 0.7748 sec Train loss: 4.4470
Output: tyrion said . he was a man ' s landing . the king ' s watch was a man ' s landing . the king ' s watch was a man ' s landing . the king ' s watch was a man ' s landing . the king ' s watch was a man ' s landing . the king ' s

Time for epoch 40 is 0.7742 sec Train loss: 3.9744
Output: tyrion said . i ' m not . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Time for epoch 60 is 0.7763 sec Train loss: 3.6375
Output: tyrion said . i ' ll be a man to be a child , but i ' m not , i ' ll be a man to be a child , but i ' m not , i ' ll be a man to be a child , but i ' m not , i ' ll be a man to be a

Time for epoch 80 is 0

## Ejercicio
- Incrementar el tamaño del dataset utilizando todos los libros de _A song of ice and fire_.
- Remplazar GRU por LSTM.
- Entrenar con diferentes métodos de de Tokenización.